In [ ]:
from __future__ import print_function
from IPython.core.display import display, HTML
from tqdm import tqdm_notebook as tqdm
import time
import pickle
from functools import reduce
import warnings
import cubic_spline_planner
import math
import copy
from matplotlib import patches
import matplotlib.pyplot as plt
import numpy as np
import pymp
%matplotlib notebook

display(HTML("<style>.container { width:80% !important; }</style>"))

warnings.filterwarnings('ignore')

# Trajectory Generation Parameters

# maximum speed [m/s]
max_speed_m_s = 4.0
# maximum lateral acceleration [m/ss]
max_lat_acc_m_s_2 = 100.0
# maximum longitudinal acceleration [m/ss]
max_long_acc_m_s_2 = 1.0
# maximum longitudinal deceleration [m/ss]
max_long_dec_m_s_2 = 1.0
# maximum curvature [1/m]
max_curvature_1_m = 0.15
# maximum road width [m]
max_road_width_m = 0.0
# road width sampling length [m]
road_width_m = 2.0
# Dynamic programming time step [s]
dyn_prog_time_step_s = 3.0
# trajectory sampling time [s]
traj_sampling_time_s = 1.0
# max prediction time [s]
max_predict_time_s = 18.0
# min prediction time [s]
min_predict_time_s = 6.0
# target speed [m/s]
target_speed_m_s = 4.0
# speed sampling length [m/s]
speed_sampling_time_s = 0.8
# number of speed sample samples
num_speed_sample = 5
# avoidance vehicle radius [m]
avoid_vehicle_radius_m = 4.0
# stopping vehicle radius [m]
stop_vehicle_radius_m = 10.0
# length zero threshold (values less than this assumed zero)
length_zero_thr_m = 0.6
# minimum trajectory point speed
min_speed_m_s = 0.5
# trajectory stitching transition rate (higher value = faster transition)
stitch_transition_rate = 5.0

# jerk cost weight
jerk_cost_weight = 0.0
# time cost weight
time_cost_weight = 0.0
# final-state cost weight
final_state_cost_weight = 1.0
# lateral cost weight
lateral_cost_weight = 1.0
# longitudinal cost weight
long_cost_weight = 10.0


# node sampling time
traj_generator_samlple_time_s = 1.0
vehicle_radius_m = stop_vehicle_radius_m


show_animation = True


class quintic_polynomial:

    def __init__(self, xs, vxs, axs, xe, vxe, axe, T):
        # calc coefficient of quintic polynomial
        self.a0 = xs
        self.a1 = vxs
        self.a2 = axs / 2.0

        A = np.array([[T ** 3, T ** 4, T ** 5],
                      [3 * T ** 2, 4 * T ** 3, 5 * T ** 4],
                      [6 * T, 12 * T ** 2, 20 * T ** 3]])
        b = np.array([xe - self.a0 - self.a1 * T - self.a2 * T ** 2,
                      vxe - self.a1 - 2 * self.a2 * T,
                      axe - 2 * self.a2])
        x = np.linalg.solve(A, b)

        self.a3 = x[0]
        self.a4 = x[1]
        self.a5 = x[2]

    def calc_point(self, t):
        xt = self.a0 + self.a1 * t + self.a2 * t ** 2 + \
            self.a3 * t ** 3 + self.a4 * t ** 4 + self.a5 * t ** 5

        return xt

    def calc_first_derivative(self, t):
        xt = self.a1 + 2 * self.a2 * t + \
            3 * self.a3 * t ** 2 + 4 * self.a4 * t ** 3 + 5 * self.a5 * t ** 4

        return xt

    def calc_second_derivative(self, t):
        xt = 2 * self.a2 + 6 * self.a3 * t + 12 * \
            self.a4 * t ** 2 + 20 * self.a5 * t ** 3

        return xt

    def calc_third_derivative(self, t):
        xt = 6 * self.a3 + 24 * self.a4 * t + 60 * self.a5 * t ** 2

        return xt


class quartic_polynomial:

    def __init__(self, xs, vxs, axs, vxe, axe, T):
        # calc coefficient of quartic polynomial

        self.a0 = xs
        self.a1 = vxs
        self.a2 = axs / 2.0

        A = np.array([[3 * T ** 2, 4 * T ** 3],
                      [6 * T, 12 * T ** 2]])
        b = np.array([vxe - self.a1 - 2 * self.a2 * T,
                      axe - 2 * self.a2])
        x = np.linalg.solve(A, b)

        self.a3 = x[0]
        self.a4 = x[1]

    def calc_point(self, t):
        xt = self.a0 + self.a1 * t + self.a2 * t ** 2 + \
            self.a3 * t ** 3 + self.a4 * t ** 4

        return xt

    def calc_first_derivative(self, t):
        xt = self.a1 + 2 * self.a2 * t + \
            3 * self.a3 * t ** 2 + 4 * self.a4 * t ** 3

        return xt

    def calc_second_derivative(self, t):
        xt = 2 * self.a2 + 6 * self.a3 * t + 12 * self.a4 * t ** 2

        return xt

    def calc_third_derivative(self, t):
        xt = 6 * self.a3 + 24 * self.a4 * t

        return xt


class GlobalPath:
    def __init__(self, x_m=None, y_m=None, yaw_rad=None, c_1_m=None, v_m_s=None, a_m_s_2=None):
        self.x_m = x_m or []
        self.y_m = y_m or []
        self.yaw_rad = yaw_rad or []
        self.c_1_m = c_1_m or []
        self.v_m_s = v_m_s or []
        self.a_m_s_2 = a_m_s_2 or []


class FrenetPath:
    def __init__(self, t_s=None,
                 d_m=None, d_d_m_s=None, d_dd_m_s_2=None, d_ddd_m_s_3=None,
                 s_m=None, s_d_m_s=None, s_dd_m_s_2=None, s_ddd_m_s_3=None,
                 t_offset_s=0, cd=0, cv=0, cf=0):
        self.t_s = t_s or []
        self.d_m = d_m or []
        self.d_d_m_s = d_d_m_s or []
        self.d_dd_m_s_2 = d_dd_m_s_2 or []
        self.d_ddd_m_s_3 = d_ddd_m_s_3 or []
        self.s_m = s_m or []
        self.s_d_m_s = s_d_m_s or []
        self.s_dd_m_s_2 = s_dd_m_s_2 or []
        self.s_ddd_m_s_3 = s_ddd_m_s_3 or []
        self.t_offset_s = t_offset_s or 0
        self.cd = cd or 0
        self.cv = cv or 0
        self.cf = cf or 0

    def __add__(self, o):
        return FrenetPath(
            self.t_s + [e + o.t_offset_s for e in o.t_s[1:]],
            self.d_m + o.d_m[1:],
            self.d_d_m_s + o.d_d_m_s[1:],
            self.d_dd_m_s_2 + o.d_dd_m_s_2[1:],
            self.d_ddd_m_s_3 + o.d_ddd_m_s_3[1:],
            self.s_m + o.s_m[1:],
            self.s_d_m_s + o.s_d_m_s[1:],
            self.s_dd_m_s_2 + o.s_dd_m_s_2[1:],
            self.s_ddd_m_s_3 + o.s_ddd_m_s_3[1:],
            self.t_offset_s,
            self.cd + o.cd,
            self.cv + o.cv,
            self.cf + o.cf)

    def stitch(self, o, ind_offset):
        return FrenetPath(
            self.t_s,
            FrenetPath._stitch(self.d_m, o.d_m, ind_offset),
            FrenetPath._stitch(self.d_d_m_s, o.d_d_m_s, ind_offset),
            FrenetPath._stitch(self.d_dd_m_s_2, o.d_dd_m_s_2, ind_offset),
            FrenetPath._stitch(self.d_ddd_m_s_3, o.d_ddd_m_s_3, ind_offset),
            FrenetPath._stitch(self.s_m, o.s_m, ind_offset),
            FrenetPath._stitch(self.s_d_m_s, o.s_d_m_s, ind_offset),
            FrenetPath._stitch(self.s_dd_m_s_2, o.s_dd_m_s_2, ind_offset),
            FrenetPath._stitch(self.s_ddd_m_s_3, o.s_ddd_m_s_3, ind_offset),
            self.t_offset_s,
            self.cd,
            self.cv,
            self.cf)

    def get_global_path(self, ref_line):
        gp = GlobalPath()
        for i in range(len(self.s_m)):
            # position
            success, (x_r, y_r) = ref_line.calc_position(self.s_m[i])
            if not success:
                break
            theta_r = ref_line.calc_yaw(self.s_m[i])
            d = self.d_m[i]
            x_x = x_r + d * math.cos(theta_r + math.pi / 2.0)
            y_x = y_r + d * math.sin(theta_r + math.pi / 2.0)
            gp.x_m.append(x_x)
            gp.y_m.append(y_x)

            # yaw
            k_r = ref_line.calc_curvature(self.s_m[i])
            d_d = self.d_d_m_s[i]
            A = 1 - k_r * d
            B = 1 / A
            tan_delta_th = d_d * B
            theta_x = np.arctan(tan_delta_th) + theta_r
            gp.yaw_rad.append(theta_x)

            # curvature
            d_dd = self.d_dd_m_s_2[i]
            k_d_r = ref_line.calc_curvature_derivative(self.s_m[i])
            C = 1 / (1 + tan_delta_th ** 2)
            D = np.sqrt(C)
            delta_theta_d = (d_dd + (k_d_r * d + k_r * d_d) *
                             tan_delta_th) * C * B
            k_x = D * B * (delta_theta_d + k_r)
            gp.c_1_m.append(k_x)

            # velocity
            s_dot = self.s_d_m_s[i]
            v_x = s_dot * A / D
            gp.v_m_s.append(v_x)

            # acceleration
            s_ddot = self.s_dd_m_s_2[i]
            a_x = s_ddot * A / D + s_dot ** 2 / D * \
                (A * tan_delta_th * delta_theta_d - (k_d_r * d + k_r * d_d))
            gp.a_m_s_2.append(a_x)

        return gp

    def _stitch(new, old, ind_offset):
        old_overlap = old[ind_offset:]
        min_len = min(len(old_overlap), len(new))
        old_overlap = old_overlap[:min_len]
        new_overlap = new[:min_len]
        x = np.linspace(0, 1, min_len)
        y = 1 / (1 + np.exp(-x * stitch_transition_rate +
                            stitch_transition_rate / 2))
        tmp = new_overlap * y + old_overlap * (1 - y)

        return list(tmp) + new[len(tmp):]


class FrenetCalcData:
    def __init__(self, c_s=None, c_s_d=None, c_s_dd=None, c_d=None, c_d_d=None, c_d_dd=None, t_f=None, s_f=None, d_f=None,
                 v_f=None, t_offset=None, maneuver=None, long_obj=None):
        self.c_s = c_s  # current 's'
        self.c_s_d = c_s_d  # current 's' first time derivative
        self.c_s_dd = c_s_dd  # current 's' second time derivative
        self.c_d = c_d  # current 'd'c
        self.c_d_d = c_d_d  # current 'd' first time derivative
        self.c_d_dd = c_d_dd  # current 'd' second time derivative
        self.t_f = t_f  # final time (path time duration)
        self.s_f = s_f  # final 's'
        self.d_f = d_f  # final 'd'
        self.v_f = v_f  # final 'v'
        self.t_offset = t_offset  # time_offset of frenet path
        self.maneuver = maneuver # manuever
        self.long_obj = long_obj # long_cost


def calc_frenet_path(f_data: FrenetCalcData):
    f_path = FrenetPath()

    f_path.t_s = [t for t in np.linspace(0.0, f_data.t_f, round(
        f_data.t_f / traj_sampling_time_s) + 1)]

    if f_data.maneuver == 'keep_speed':
        lon_qp = quartic_polynomial(f_data.c_s, f_data.c_s_d, f_data.c_s_dd, f_data.v_f,
                                    0.0, f_data.t_f)
    elif f_data.maneuver == 'stop':
        lon_qp = quintic_polynomial(f_data.c_s, f_data.c_s_d, f_data.c_s_dd, f_data.s_f,
                                    0.0, 0.0,  f_data.t_f)
    else:
        raise Exception('maneuver is not defined.')
        
    f_path.s_m = [lon_qp.calc_point(t) for t in f_path.t_s]
    f_path.s_d_m_s = [lon_qp.calc_first_derivative(t) for t in f_path.t_s]
    f_path.s_dd_m_s_2 = [lon_qp.calc_second_derivative(t) for t in f_path.t_s]
    f_path.s_ddd_m_s_3 = [lon_qp.calc_third_derivative(t) for t in f_path.t_s]

    if f_path.s_m[-1] - f_path.s_m[0] > np.finfo(float).eps:
        lat_qp = quintic_polynomial(f_data.c_d, f_data.c_d_d, f_data.c_d_dd, f_data.d_f,
                                    0.0, 0.0, f_path.s_m[-1] - f_path.s_m[0])

        f_path.d_m = [lat_qp.calc_point(t - f_path.s_m[0]) for t in f_path.s_m]
        f_path.d_d_m_s = [lat_qp.calc_first_derivative(
            t - f_path.s_m[0]) for t in f_path.s_m]
        f_path.d_dd_m_s_2 = [lat_qp.calc_second_derivative(
            t - f_path.s_m[0]) for t in f_path.s_m]
        f_path.d_ddd_m_s_3 = [lat_qp.calc_third_derivative(
            t - f_path.s_m[0]) for t in f_path.s_m]
    else:
        f_path.d_m = [f_data.c_d for t in f_path.s_m]
        f_path.d_d_m_s = [f_data.c_d_d for t in f_path.s_m]
        f_path.d_dd_m_s_2 = [f_data.c_d_dd for t in f_path.s_m]
        f_path.d_ddd_m_s_3 = [0 for t in f_path.s_m]

    Jp = np.sum(np.power(f_path.d_ddd_m_s_3, 2))  # square of jerk
    Js = np.sum(np.power(f_path.s_ddd_m_s_3, 2))  # square of jerk


    if f_data.long_obj == 'speed':
        # square of diff from target speed
        ds = np.sum((target_speed_m_s - np.array(f_path.s_d_m_s)) ** 2)
    elif f_data.long_obj == 'position':
        # square of diff from target 's'
        ds = np.sum((f_data.s_f - np.array(f_path.s_m)) ** 2)
    else:
        raise Exception('longitudinal cost is not defined.')

    f_path.cd = (jerk_cost_weight * Jp) + (time_cost_weight *
                                           f_data.t_f) + (final_state_cost_weight * f_path.d_m[-1] ** 2)
    f_path.cv = (jerk_cost_weight * Js) + (time_cost_weight *
                                           f_data.t_f) + (final_state_cost_weight * ds)
    f_path.cf = (lateral_cost_weight * f_path.cd) + \
                (long_cost_weight * f_path.cv)

    if (not (f_path.s_m[-1] - f_path.s_m[0] > 0)) and abs(f_data.c_d - f_data.d_f) > np.finfo(float).eps:
        f_path.cd = np.inf
        f_path.cf = np.inf

    f_path.t_offset_s = f_data.t_offset

    return f_path


def normalize_angle(x):
    pi_mul_2 = 2.0 * np.pi
    retval = x - (pi_mul_2 * np.floor(x / (pi_mul_2)))
    if retval > np.pi:
        retval -= pi_mul_2
    return retval


def check_trajectory(frenet_path: FrenetPath, global_path: GlobalPath, objects, dynamic_objects):
    feasible = True
    if len(frenet_path.s_m) != len(global_path.x_m):
        feasible = False
    if any([v > max_speed_m_s + np.finfo(float).eps for v in global_path.v_m_s]):  # Max speed check
        feasible = False
    elif any([v < -np.finfo(float).eps for v in global_path.v_m_s]):  # Negative speed check
        feasible = False
    elif any([a > max_long_acc_m_s_2 + np.finfo(float).eps for a in global_path.a_m_s_2]):  # Max acceleration check
        feasible = False
    elif any([-a > max_long_dec_m_s_2 + np.finfo(float).eps for a in global_path.a_m_s_2]):  # Max deceleration check
        feasible = False
    elif any([lat_a > max_lat_acc_m_s_2 + np.finfo(float).eps for lat_a in np.abs(np.array(global_path.c_1_m)) * np.array(global_path.v_m_s)*np.array(global_path.v_m_s)]):  # Max deceleration check
        feasible = False
    elif any([abs(c) > max_curvature_1_m + np.finfo(float).eps for c in global_path.c_1_m]):  # Max curvature check
        feasible = False
    for i in range(len(objects[:, 0])):
        d = [((ix - objects[i, 0]) ** 2 + (iy - objects[i, 1]) ** 2)
             for (ix, iy) in zip(global_path.x_m, global_path.y_m)]

        if any([di + np.finfo(float).eps < vehicle_radius_m ** 2 for di in d]):
            feasible = False

    if not feasible:
        frenet_path.cf = np.inf


def calc_and_check_frenet_path(ref_line_spline, objects, dynamic_objects, f_data):
    f_path = calc_frenet_path(f_data)
    g_path = f_path.get_global_path(ref_line_spline)
    check_trajectory(
        f_path, g_path, objects, dynamic_objects)
    return f_path


def get_optimal_path(ref_line_spline, objects, dynamic_objects, c_s, c_s_d, c_s_dd, c_d, c_d_d, c_d_dd):
    d_vec = np.linspace(-max_road_width_m, max_road_width_m,
                        round(2 * max_road_width_m / road_width_m + 1))
    v_vec = np.linspace(0, num_speed_sample *
                        speed_sampling_time_s, num_speed_sample + 1)

    dyn_prog = [[[{'cost': np.inf, 'path': [], 's': c_s} for e1 in range(
        len(v_vec))] for e2 in range(len(d_vec))] for e3 in range(round(max_predict_time_s / dyn_prog_time_step_s + 1))]
    for d_ind, d in enumerate(d_vec):
        for v_ind, v in enumerate(v_vec):
            dyn_prog[0][d_ind][v_ind]['cost'] = 0

    s_f = ref_line_spline.s[-1]
    stop_mode = ((s_f - c_s) < 2 * target_speed_m_s **2/2/max_long_dec_m_s_2)
    if stop_mode:
        stop_speed_sampling_time_s = c_s_d / float(num_speed_sample)
        v_vec = np.linspace(0, num_speed_sample *
                        stop_speed_sampling_time_s, num_speed_sample + 1)
    for dyn_ind in range(round(min_predict_time_s / dyn_prog_time_step_s),
                         round(max_predict_time_s / dyn_prog_time_step_s + 1)):
        maneuver = 'stop' if stop_mode and dyn_ind == round(max_predict_time_s / dyn_prog_time_step_s) else 'keep_speed'
        partial_update = pymp.shared.list(range(len(d_vec) * len(v_vec)))
        with pymp.Parallel(6) as p:
            for d2_ind in p.range(0, len(d_vec)):
                d2 = d_vec[d2_ind]
                for v2_ind, v2 in enumerate(v_vec):
                    min_cost = np.inf
                    optim_seq = []
                    optim_s = 0
                    for t_ind in range(0, dyn_ind - round(min_predict_time_s / dyn_prog_time_step_s) + 1):
                        t = (dyn_ind - t_ind) * dyn_prog_time_step_s
                        t_offset = t_ind * dyn_prog_time_step_s
                        if t_ind == 0:
                            f_data = FrenetCalcData(c_s=c_s, c_s_d=c_s_d, c_s_dd=c_s_dd,
                                                    c_d=c_d, c_d_d=c_d_d, c_d_dd=c_d_dd, t_f=t, d_f=d2, v_f=v2,
                                                    t_offset=t_offset, maneuver=maneuver, long_obj='position', s_f= s_f)
                            f_path = calc_and_check_frenet_path(
                                ref_line_spline, objects, dynamic_objects, f_data)
                            new_cost = f_path.cf
                            if new_cost < min_cost:
                                min_cost = new_cost
                                optim_seq = [(t, 0, 0, d2, v2)]
                                optim_s = f_path.s_m[-1]
                        else:
                            for d1_ind, d1 in enumerate(d_vec):
                                for v1_ind, v1 in enumerate(v_vec):
                                    dyn_s0 = dyn_prog[t_ind][d1_ind][v1_ind]['s']
                                    f_data = FrenetCalcData(c_s=dyn_s0, c_s_d=v1, c_s_dd=0.0,
                                                            c_d=d1, c_d_d=0.0, c_d_dd=0.0, t_f=t, d_f=d2, v_f=v2,
                                                            t_offset=t_offset, maneuver=maneuver, long_obj='position', s_f= s_f)
                                    f_path = calc_and_check_frenet_path(
                                        ref_line_spline, objects, dynamic_objects, f_data)
                                    new_cost = dyn_prog[t_ind][d1_ind][v1_ind]['cost'] + f_path.cf
                                    if new_cost < min_cost:
                                        min_cost = new_cost
                                        optim_seq = dyn_prog[t_ind][d1_ind][v1_ind]['path'] + [
                                            (t, d1, v1, d2, v2)]
                                        optim_s = f_path.s_m[-1]

                    partial_update[v2_ind + d2_ind *
                                   len(v_vec)] = (min_cost, optim_seq, optim_s)

        for d2_ind in range(len(d_vec)):
            for v2_ind in range(len(v_vec)):
                dyn_prog[dyn_ind][d2_ind][v2_ind]['cost'] = partial_update[v2_ind +
                                                                           d2_ind * len(v_vec)][0]
                dyn_prog[dyn_ind][d2_ind][v2_ind]['path'] = partial_update[v2_ind +
                                                                           d2_ind * len(v_vec)][1]
                dyn_prog[dyn_ind][d2_ind][v2_ind]['s'] = partial_update[v2_ind +
                                                                        d2_ind * len(v_vec)][2]

    final_min_cost = np.inf
    best_sequence = None
    for d_ind, d in enumerate(d_vec):
        for v_ind, v in enumerate(v_vec):
            if dyn_prog[-1][d_ind][v_ind]['cost'] < final_min_cost:
                final_min_cost = dyn_prog[-1][d_ind][v_ind]['cost']
                best_sequence = dyn_prog[-1][d_ind][v_ind]['path']

    if best_sequence is None:
        return None
    maneuver = 'stop' if stop_mode and len(best_sequence) == 1 else 'keep_speed' 
    segment = best_sequence[0]
    f_data = FrenetCalcData(c_s=c_s, c_s_d=c_s_d, c_s_dd=c_s_dd,
                            c_d=c_d, c_d_d=c_d_d, c_d_dd=c_d_dd, t_f=segment[
                                0], d_f=segment[3], v_f=segment[4],
                            t_offset=0, maneuver=maneuver, long_obj='position', s_f= s_f)
    segments = [calc_frenet_path(f_data)]
    s = segments[-1].s_m[-1]
    t = segments[-1].t_s[-1]
    stopping_trajectory = False
    for i, segment in enumerate(best_sequence[1:]):
        maneuver = 'stop' if stop_mode and i == len(best_sequence) - 2 else 'keep_speed' 
        v_f = segment[4]
        f_data = FrenetCalcData(c_s=s, c_s_d=segment[2], c_s_dd=0.0,
                                c_d=segment[1], c_d_d=0.0, c_d_dd=0.0, t_f=segment[0], d_f=segment[3], v_f=v_f,
                                t_offset=t, maneuver=maneuver, long_obj='position', s_f= s_f)
        fp = calc_frenet_path(f_data)
        segments.append(fp)
        s = fp.s_m[-1]
        t += fp.t_s[-1]

    return reduce((lambda x, y: x + y), segments)


def generate_target_course(x, y):
    ref_line = cubic_spline_planner.Spline2D(x, y)
    s = np.arange(0, ref_line.s[-1], 0.1)

    rx, ry, ryaw, rk = [], [], [], []
    for i_s in s:
        result, (ix, iy) = ref_line.calc_position(i_s)
        rx.append(ix)
        ry.append(iy)
        ryaw.append(ref_line.calc_yaw(i_s))
        rk.append(ref_line.calc_curvature(i_s))

    return rx, ry, ryaw, rk, ref_line


# way points
(wx, wy) = pickle.load(open('ref_line4.pkl', 'rb'))

tx, ty, tyaw, tc, ref_line_spline = generate_target_course(wx, wy)

L = 1.5

# initial state
c_s = 0.0  # current course position
c_s_d = 0.0  # current speed [m/s]
c_s_dd = 0.0  # current speed [m/s]
x = wx[0]
y = wy[0]
c_d = 0.0  # current lateral position [m]
c_d_d = 0  # current lateral speed [m/s]
c_d_dd = 0  # current lateral speed [m/s]

area = 30.0  # animation area length [m]

SIM_TIME = 86
prev_traj_samp_time = -traj_generator_samlple_time_s
prev_anim_time = 0.0
anim_time = traj_sampling_time_s
t = 0.0
frame_num = 0

objects = np.array([[1000, 1000]])
dyn_objects = []

if show_animation:
    # initialise the graph and settings
    fig = plt.figure()
    ax = fig.gca()
    fig.show()
    fig.canvas.draw()

prev_f_path = None
f_path_arr = []
v_arr = []
t_arr = []
for i in tqdm(range(round(SIM_TIME / traj_sampling_time_s))):
    t_arr.append(t)
    v_arr.append(c_s_d)

    if (t >= prev_traj_samp_time + traj_generator_samlple_time_s):
        f_path = get_optimal_path(
            ref_line_spline, objects, dyn_objects, c_s, c_s_d, c_s_dd, c_d, c_d_d, c_d_dd)
        if f_path is None:
            print("No feasible path!")        
            #assert(False)

    f_path_arr.append(f_path)
    g_path = f_path.get_global_path(ref_line_spline)
    ind = 1

    c_s = f_path.s_m[ind]
    c_s_d = f_path.s_d_m_s[ind]
    c_s_dd = f_path.s_dd_m_s_2[ind]
    c_d = f_path.d_m[ind]
    c_d_d = f_path.d_d_m_s[ind]
    c_d_dd = f_path.d_dd_m_s_2[ind]
    x = g_path.x_m[ind]
    y = g_path.y_m[ind]
    t += traj_sampling_time_s

    if show_animation and t >= prev_anim_time + anim_time:  # pragma: no cover
        prev_anim_time = t
        ax.clear()
        # for stopping simulation with the esc key.
        ax.set_aspect('equal', adjustable='box')
        ax.plot(wx, wy, '.')
        ax.plot(tx, ty)
        ax.plot(objects[:, 0], objects[:, 1], "xk")
        ax.plot(g_path.x_m, g_path.y_m, "-or")
        ax.plot(g_path.x_m[ind], g_path.y_m[ind], "-oy")

        # ax.plot(x, y, ".c")
        rect = patches.Rectangle(
            (x, y), 2.5, 1, g_path.yaw_rad[ind] * 180 / np.pi, fill=True)
        ax.add_patch(rect)
        rect = patches.Rectangle(
            (x, y), -2.5, 1, g_path.yaw_rad[ind] * 180 / np.pi, fill=False)
        ax.add_patch(rect)
        rect = patches.Rectangle(
            (x, y), 2.5, -1, g_path.yaw_rad[ind] * 180 / np.pi, fill=True)
        ax.add_patch(rect)
        rect = patches.Rectangle(
            (x, y), -2.5, -1, g_path.yaw_rad[ind] * 180 / np.pi, fill=False)
        ax.add_patch(rect)

        plt.xlim(x - area, x + area)
        plt.ylim(y - area, y + area)
        plt.title("v[m/s]:" + '%.2f' % c_s_d + " s[m]:" + '%.2f' %
                  c_s + ' t[s]:' + '%.2f' % t)
        ax.grid(True)
        fig.canvas.draw()  # draw
        # fig.savefig(str(frame_num)+'.jpg')
        frame_num += 1

    time.sleep(0.001)  # sleep

print("Finish")


In [ ]:
s = np.linspace(0,180,400)
k = np.array([ref_line_spline.calc_curvature(e) for e in s])
kd = np.array([ref_line_spline.calc_curvature_derivative(e) for e in s])
ya = np.array([ref_line_spline.calc_yaw(e) for e in s])

plt.figure(figsize=(12,5))
plt.plot(s,k,'.-')
#plt.plot(s,kd,'g.-')
plt.plot(f_path.s_m, np.array(g_path.c_1_m),'r.-')
plt.grid()

plt.figure(figsize=(12,5))
plt.plot(s,ya*180/np.pi,'.-')
#plt.plot(s,kd,'g.-')
plt.plot(f_path.s_m, np.array(g_path.yaw_rad)*180/np.pi,'r.-')
plt.grid()

k = np.array([list(ref_line_spline.calc_position(e)[1]) for e in s])
plt.figure()
plt.plot(k[:,0], k[:,1],'.-')
plt.plot(g_path.x_m, g_path.y_m,'r')
plt.gca().set_aspect('equal', adjustable='box')
plt.grid()


In [ ]:
from sympy import *
x, y, s = symbols('x y s')
x = Function('x')
y = Function('y')
dx = x(s).diff(s)
dy = y(s).diff(s)
ddx = dx.diff(s)
ddy = dy.diff(s)
k = (ddy * dx - ddx * dy) / (dx ** 2 + dy ** 2) ** (3.0/2.0)

print(simplify(diff(k,s)))
simplify(diff(k,s))

In [ ]:
c_s_dd = 0
c_d_dd = 0

c_s, c_s_d, c_d, c_d_d = 2.5, 0, 6.50094e-15, 0


fp = get_optimal_path(ref_line_spline, objects, dyn_objects, c_s, c_s_d, c_s_dd, c_d, c_d_d, c_d_dd)

In [ ]:
lines = []
fp_arr = []
obj_arr = []
with open('/home/hamid/adehome/safeai/log_file.txt') as log_file:
    for line in log_file:
        lines.append(line)

for i in range(int(len(lines)/7)):
    print(i, '/', len(lines)/7)
    c_s, c_s_d, c_s_dd, c_d, c_d_d, c_d_dd = tuple([float(e) for e in lines[i*7].strip().split(',')])
    obj_xy = [float(e) for e in lines[i*7+1].strip().split(',')[:-1]]
    objects = np.array(list(zip(obj_xy[::2],obj_xy[1::2])))
    if len(objects) == 0:
        objects = np.array([[10000,10000]])
    t = [float(e) for e in lines[i*7+2].strip().split(',')[:-1]]
    s = [float(e) for e in lines[i*7+3].strip().split(',')[:-1]]
    s_d = [float(e) for e in lines[i*7+4].strip().split(',')[:-1]]
    d = [float(e) for e in lines[i*7+5].strip().split(',')[:-1]]

    fp = get_optimal_path(ref_line_spline, objects, dyn_objects, c_s, c_s_d, c_s_dd, c_d, c_d_d, c_d_dd)
    fp_arr.append(fp)
    obj_arr.append(objects)

    print(c_s, c_s_d, c_s_dd, c_d, c_d_d, c_d_dd)
    print(t)
    print(s)
    print(s_d)
    print(d)
    print()
    if fp is not None:
        print(fp.t_s)
        print(fp.s_m)
        print(fp.s_d_m_s)
        print(fp.d_m)

        assert np.max(np.abs(np.array(fp.t_s) - np.array(t))) < 0.001
        assert np.max(np.abs(np.array(fp.s_m) - np.array(s))) < 0.001
        assert np.max(np.abs(np.array(fp.d_m) - np.array(d))) < 0.001
    else:
        print("[-1.0]")
        print("[-1.0]")
        print("[-1.0]")

        assert t[0] == -1

In [ ]:
for i in range(len(fp_arr)):
    if (fp_arr[i] is not None):
        g_path = fp_arr[i].get_global_path(ref_line_spline)
        plt.figure()
        plt.plot(wx, wy, '.')
        if obj_arr[i][0,0] < 1000:
            plt.plot(obj_arr[i][:, 0], obj_arr[i][:, 1], "xk")
        plt.plot(g_path.x_m, g_path.y_m,'.')
        plt.gca().set_aspect('equal', adjustable='box')